In [21]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Load sequences and labels
sequences_path = 'C:/Users/Rahul Ganatra/OneDrive/Desktop/frame_extraction/ProjectData/sequences.npy'
labels_path = 'C:/Users/Rahul Ganatra/OneDrive/Desktop/frame_extraction/ProjectData/labels.npy'

# Check if files exist and are correctly loaded
try:
    sequences = np.load(sequences_path)
    labels = np.load(labels_path)
    print("Sequences and labels loaded successfully.")
except Exception as e:
    print(f"Error loading files: {e}")

# Verify shapes of loaded data
print(f"Sequences shape: {sequences.shape}")
print(f"Labels shape: {labels.shape}")

# Encode labels as integers
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# Convert labels to one-hot encoded format
labels_one_hot = to_categorical(labels_encoded)

# Split the data into training and testing sets
try:
    X_train, X_test, y_train, y_test = train_test_split(sequences, labels_one_hot, test_size=0.2, random_state=42)
    print("Data split into training and testing sets successfully.")
except Exception as e:
    print(f"Error during train-test split: {e}")

# Define the LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.5),
    LSTM(64, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Sequences and labels loaded successfully.
Sequences shape: (109, 30, 63)
Labels shape: (109,)
Data split into training and testing sets successfully.


C:\Users\Rahul Ganatra\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_16 (LSTM)                  │ (None, 30, 64)         │        32,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 30, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_17 (LSTM)                  │ (None, 64)             │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 70,472 (275.28 KB)

 Trainable params: 70,472 (275.28 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
try:
    history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_data=(X_test, y_test))
    print("Model trained successfully.")
except Exception as e:
    print(f"Error during model training: {e}")
    raise



Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 185ms/step - accuracy: 0.1956 - loss: 2.0564 - val_accuracy: 0.1818 - val_loss: 2.0528
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.2845 - loss: 1.9765 - val_accuracy: 0.1818 - val_loss: 2.0704
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.1448 - loss: 2.0777 - val_accuracy: 0.0909 - val_loss: 2.0686
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.2186 - loss: 1.9518 - val_accuracy: 0.0909 - val_loss: 2.0917
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.1625 - loss: 1.9779 - val_accuracy: 0.1364 - val_loss: 2.1021
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.1432 - loss: 1.9332 - val_accuracy: 0.1818 - val_loss: 2.1125
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.3542 - loss: 1.9470 - val_accuracy: 0.1818 - val_loss: 2.1198
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.2066 - loss: 1.9301 - val_accuracy: 0.3182 - val_loss: 2.1223

In [23]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.3182 - loss: 1.8156
Test Loss: 1.8156256675720215
Test Accuracy: 0.3181818127632141
